In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from matplotlib.gridspec import GridSpec
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 10
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load Data

In [22]:
def load_data():
    """Load all three CSV files and combine them"""

    # Load GPT-2 results
    gpt2_df = pd.read_csv('/content/drive/MyDrive/Carbonemission/Version2/gpt2_squadv2_results.csv')
    gpt2_df['dataset_size%'] = gpt2_df['dataset_fraction'] * 100

    # Load BERT results
    bert_df = pd.read_csv('/content/drive/MyDrive/Carbonemission/Version2/bert_dataset_size_results.csv')

    # Load DistilBERT results
    distilbert_df = pd.read_csv('/content/drive/MyDrive/Carbonemission/Version2/distilbert_dataset_size_results (1).csv')

    # Standardize column names if needed
    for df in [gpt2_df, bert_df, distilbert_df]:
        if 'model_name' in df.columns:
            df['model'] = df['model_name']
        df['dataset_size'] = df['dataset_size%']

    # Combine all dataframes
    combined_df = pd.concat([
        gpt2_df[['model', 'dataset_size', 'f1_score', 'exact_match',
                 'emissions_kg', 'energy_consumed_kwh', 'training_time_hours',
                 'cpu_power_w', 'gpu_power_w', 'ram_power_w']],
        bert_df[['model_name', 'dataset_size%', 'f1_score', 'exact_match',
                 'emissions_kg', 'energy_consumed_kwh', 'training_time_hours',
                 'cpu_power_w', 'gpu_power_w', 'ram_power_w']].rename(
                     columns={'model_name': 'model', 'dataset_size%': 'dataset_size'}),
        distilbert_df[['model_name', 'dataset_size%', 'f1_score', 'exact_match',
                       'emissions_kg', 'energy_consumed_kwh', 'training_time_hours',
                       'cpu_power_w', 'gpu_power_w', 'ram_power_w']].rename(
                           columns={'model_name': 'model', 'dataset_size%': 'dataset_size'})
    ], ignore_index=True)

    return combined_df, gpt2_df, bert_df, distilbert_df

## Performance Metrics Comparison

In [23]:
def plot_performance_comparison(df):
    """Plot F1 Score and Exact Match comparison"""
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))

    dataset_sizes = sorted(df['dataset_size'].unique())
    models = df['model'].unique()
    colors = sns.color_palette("husl", len(models))

    # F1 Score comparison
    ax1 = axes[0]
    for idx, model in enumerate(models):
        model_data = df[df['model'] == model].sort_values('dataset_size')
        ax1.plot(model_data['dataset_size'], model_data['f1_score'],
                marker='o', linewidth=2.5, markersize=10, label=model,
                color=colors[idx])

    ax1.set_xlabel('Dataset Size (%)', fontsize=12, fontweight='bold')
    ax1.set_ylabel('F1 Score', fontsize=12, fontweight='bold')
    ax1.set_title('F1 Score vs Dataset Size', fontsize=14, fontweight='bold', pad=20)
    ax1.legend(fontsize=11, frameon=True, shadow=True)
    ax1.grid(True, alpha=0.3)
    ax1.set_xticks(dataset_sizes)

    # Add value labels
    for idx, model in enumerate(models):
        model_data = df[df['model'] == model].sort_values('dataset_size')
        for x, y in zip(model_data['dataset_size'], model_data['f1_score']):
            ax1.annotate(f'{y:.1f}', (x, y), textcoords="offset points",
                        xytext=(0,10), ha='center', fontsize=9)

    # Exact Match comparison
    ax2 = axes[1]
    for idx, model in enumerate(models):
        model_data = df[df['model'] == model].sort_values('dataset_size')
        ax2.plot(model_data['dataset_size'], model_data['exact_match'],
                marker='s', linewidth=2.5, markersize=10, label=model,
                color=colors[idx])

    ax2.set_xlabel('Dataset Size (%)', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Exact Match Score', fontsize=12, fontweight='bold')
    ax2.set_title('Exact Match vs Dataset Size', fontsize=14, fontweight='bold', pad=20)
    ax2.legend(fontsize=11, frameon=True, shadow=True)
    ax2.grid(True, alpha=0.3)
    ax2.set_xticks(dataset_sizes)

    # Add value labels
    for idx, model in enumerate(models):
        model_data = df[df['model'] == model].sort_values('dataset_size')
        for x, y in zip(model_data['dataset_size'], model_data['exact_match']):
            ax2.annotate(f'{y:.1f}', (x, y), textcoords="offset points",
                        xytext=(0,10), ha='center', fontsize=9)

    plt.tight_layout()
    return fig

## Carbon Emissions Analysis

In [24]:
def plot_emissions_analysis(df):
    """Plot carbon emissions and energy consumption"""
    fig = plt.figure(figsize=(18, 10))
    gs = GridSpec(2, 3, figure=fig, hspace=0.3, wspace=0.3)

    models = df['model'].unique()
    colors = sns.color_palette("husl", len(models))

    # 1. Total CO2 Emissions
    ax1 = fig.add_subplot(gs[0, 0])
    x = np.arange(len(df['dataset_size'].unique()))
    width = 0.25

    for idx, model in enumerate(models):
        model_data = df[df['model'] == model].sort_values('dataset_size')
        ax1.bar(x + idx*width, model_data['emissions_kg'], width,
               label=model, color=colors[idx], alpha=0.8)

    ax1.set_xlabel('Dataset Size (%)', fontsize=11, fontweight='bold')
    ax1.set_ylabel('CO₂ Emissions (kg)', fontsize=11, fontweight='bold')
    ax1.set_title('Carbon Emissions by Model', fontsize=13, fontweight='bold')
    ax1.set_xticks(x + width)
    ax1.set_xticklabels([f'{int(s)}%' for s in sorted(df['dataset_size'].unique())])
    ax1.legend(fontsize=10)
    ax1.grid(True, alpha=0.3, axis='y')

    # 2. Energy Consumption
    ax2 = fig.add_subplot(gs[0, 1])
    for idx, model in enumerate(models):
        model_data = df[df['model'] == model].sort_values('dataset_size')
        ax2.bar(x + idx*width, model_data['energy_consumed_kwh'], width,
               label=model, color=colors[idx], alpha=0.8)

    ax2.set_xlabel('Dataset Size (%)', fontsize=11, fontweight='bold')
    ax2.set_ylabel('Energy Consumed (kWh)', fontsize=11, fontweight='bold')
    ax2.set_title('Energy Consumption by Model', fontsize=13, fontweight='bold')
    ax2.set_xticks(x + width)
    ax2.set_xticklabels([f'{int(s)}%' for s in sorted(df['dataset_size'].unique())])
    ax2.legend(fontsize=10)
    ax2.grid(True, alpha=0.3, axis='y')

    # 3. Training Time
    ax3 = fig.add_subplot(gs[0, 2])
    for idx, model in enumerate(models):
        model_data = df[df['model'] == model].sort_values('dataset_size')
        ax3.bar(x + idx*width, model_data['training_time_hours'], width,
               label=model, color=colors[idx], alpha=0.8)

    ax3.set_xlabel('Dataset Size (%)', fontsize=11, fontweight='bold')
    ax3.set_ylabel('Training Time (hours)', fontsize=11, fontweight='bold')
    ax3.set_title('Training Time by Model', fontsize=13, fontweight='bold')
    ax3.set_xticks(x + width)
    ax3.set_xticklabels([f'{int(s)}%' for s in sorted(df['dataset_size'].unique())])
    ax3.legend(fontsize=10)
    ax3.grid(True, alpha=0.3, axis='y')

    # 4. Emissions vs Dataset Size (Line plot)
    ax4 = fig.add_subplot(gs[1, 0])
    for idx, model in enumerate(models):
        model_data = df[df['model'] == model].sort_values('dataset_size')
        ax4.plot(model_data['dataset_size'], model_data['emissions_kg'],
                marker='o', linewidth=2.5, markersize=10, label=model,
                color=colors[idx])

    ax4.set_xlabel('Dataset Size (%)', fontsize=11, fontweight='bold')
    ax4.set_ylabel('CO₂ Emissions (kg)', fontsize=11, fontweight='bold')
    ax4.set_title('Emission Trends', fontsize=13, fontweight='bold')
    ax4.legend(fontsize=10)
    ax4.grid(True, alpha=0.3)

    # 5. Emissions Efficiency (F1 per kg CO2)
    ax5 = fig.add_subplot(gs[1, 1])
    for idx, model in enumerate(models):
        model_data = df[df['model'] == model].sort_values('dataset_size')
        efficiency = model_data['f1_score'] / model_data['emissions_kg']
        ax5.plot(model_data['dataset_size'], efficiency,
                marker='D', linewidth=2.5, markersize=10, label=model,
                color=colors[idx])

    ax5.set_xlabel('Dataset Size (%)', fontsize=11, fontweight='bold')
    ax5.set_ylabel('F1 Score per kg CO₂', fontsize=11, fontweight='bold')
    ax5.set_title('Carbon Efficiency (Higher is Better)', fontsize=13, fontweight='bold')
    ax5.legend(fontsize=10)
    ax5.grid(True, alpha=0.3)

    # 6. Energy Efficiency (F1 per kWh)
    ax6 = fig.add_subplot(gs[1, 2])
    for idx, model in enumerate(models):
        model_data = df[df['model'] == model].sort_values('dataset_size')
        efficiency = model_data['f1_score'] / model_data['energy_consumed_kwh']
        ax6.plot(model_data['dataset_size'], efficiency,
                marker='D', linewidth=2.5, markersize=10, label=model,
                color=colors[idx])

    ax6.set_xlabel('Dataset Size (%)', fontsize=11, fontweight='bold')
    ax6.set_ylabel('F1 Score per kWh', fontsize=11, fontweight='bold')
    ax6.set_title('Energy Efficiency (Higher is Better)', fontsize=13, fontweight='bold')
    ax6.legend(fontsize=10)
    ax6.grid(True, alpha=0.3)

    return fig

## Hardware Usage Analysis

In [25]:
def plot_hardware_usage(df):
    """Plot hardware power consumption breakdown"""
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))

    models = df['model'].unique()
    dataset_sizes = sorted(df['dataset_size'].unique())

    # Color palette
    colors = sns.color_palette("husl", len(models))

    for idx, size in enumerate(dataset_sizes):
        ax = axes[idx // 2, idx % 2]

        size_data = df[df['dataset_size'] == size]

        # Prepare data for stacked bar
        cpu_power = size_data.groupby('model')['cpu_power_w'].first()
        gpu_power = size_data.groupby('model')['gpu_power_w'].first()
        ram_power = size_data.groupby('model')['ram_power_w'].first()

        x = np.arange(len(models))
        width = 0.6

        # Create stacked bars
        p1 = ax.bar(x, cpu_power, width, label='CPU', color='#3498db', alpha=0.8)
        p2 = ax.bar(x, gpu_power, width, bottom=cpu_power, label='GPU',
                   color='#e74c3c', alpha=0.8)
        p3 = ax.bar(x, ram_power, width, bottom=cpu_power+gpu_power,
                   label='RAM', color='#2ecc71', alpha=0.8)

        ax.set_ylabel('Power Draw (W)', fontsize=11, fontweight='bold')
        ax.set_title(f'Hardware Power Usage - {int(size)}% Dataset',
                    fontsize=13, fontweight='bold')
        ax.set_xticks(x)
        ax.set_xticklabels(models, rotation=15, ha='right')
        ax.legend(fontsize=10)
        ax.grid(True, alpha=0.3, axis='y')

        # Add total values on top
        for i, model in enumerate(models):
            total = cpu_power.iloc[i] + gpu_power.iloc[i] + ram_power.iloc[i]
            ax.text(i, total + 5, f'{total:.1f}W', ha='center',
                   fontweight='bold', fontsize=9)

    plt.tight_layout()
    return fig

## Performance vs Cost Trade-off

In [26]:
def plot_performance_vs_cost(df):
    """Scatter plot showing performance vs emissions trade-off"""
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))

    models = df['model'].unique()
    colors = sns.color_palette("husl", len(models))
    markers = ['o', 's', '^']

    # F1 vs Emissions
    ax1 = axes[0]
    for idx, model in enumerate(models):
        model_data = df[df['model'] == model]
        ax1.scatter(model_data['emissions_kg'], model_data['f1_score'],
                   s=300, alpha=0.7, label=model, color=colors[idx],
                   marker=markers[idx], edgecolors='black', linewidth=1.5)

        # Add dataset size labels
        for _, row in model_data.iterrows():
            ax1.annotate(f"{int(row['dataset_size'])}%",
                        (row['emissions_kg'], row['f1_score']),
                        fontsize=9, ha='center', va='center', fontweight='bold')

    ax1.set_xlabel('CO₂ Emissions (kg)', fontsize=12, fontweight='bold')
    ax1.set_ylabel('F1 Score', fontsize=12, fontweight='bold')
    ax1.set_title('Performance vs Carbon Footprint Trade-off',
                 fontsize=14, fontweight='bold', pad=20)
    ax1.legend(fontsize=11, frameon=True, shadow=True)
    ax1.grid(True, alpha=0.3)

    # Exact Match vs Energy
    ax2 = axes[1]
    for idx, model in enumerate(models):
        model_data = df[df['model'] == model]
        ax2.scatter(model_data['energy_consumed_kwh'], model_data['exact_match'],
                   s=300, alpha=0.7, label=model, color=colors[idx],
                   marker=markers[idx], edgecolors='black', linewidth=1.5)

        # Add dataset size labels
        for _, row in model_data.iterrows():
            ax2.annotate(f"{int(row['dataset_size'])}%",
                        (row['energy_consumed_kwh'], row['exact_match']),
                        fontsize=9, ha='center', va='center', fontweight='bold')

    ax2.set_xlabel('Energy Consumed (kWh)', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Exact Match Score', fontsize=12, fontweight='bold')
    ax2.set_title('Performance vs Energy Consumption Trade-off',
                 fontsize=14, fontweight='bold', pad=20)
    ax2.legend(fontsize=11, frameon=True, shadow=True)
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    return fig

## Comprehensive Comparison Heatmap

In [27]:
def plot_comparison_heatmap(df):
    """Create a heatmap showing all metrics normalized"""
    fig, axes = plt.subplots(1, 2, figsize=(18, 6))

    # Prepare data for heatmap
    metrics = ['f1_score', 'exact_match', 'emissions_kg',
               'energy_consumed_kwh', 'training_time_hours']

    models = df['model'].unique()
    dataset_sizes = sorted(df['dataset_size'].unique())

    # Create pivot tables for each metric
    for idx, metric in enumerate(['f1_score', 'emissions_kg']):
        ax = axes[idx] if idx < 2 else axes[0]

        pivot_data = df.pivot_table(
            values=metric,
            index='model',
            columns='dataset_size'
        )

        # Normalize for better visualization (higher is better for performance)
        if metric in ['f1_score', 'exact_match']:
            cmap = 'YlGn'
            label = metric.replace('_', ' ').title()
        else:
            cmap = 'YlOrRd'
            label = metric.replace('_', ' ').title()

        sns.heatmap(pivot_data, annot=True, fmt='.2f', cmap=cmap,
                   ax=ax, cbar_kws={'label': label},
                   linewidths=1, linecolor='gray')

        ax.set_title(f'{label} by Model and Dataset Size',
                    fontsize=13, fontweight='bold', pad=20)
        ax.set_xlabel('Dataset Size (%)', fontsize=11, fontweight='bold')
        ax.set_ylabel('Model', fontsize=11, fontweight='bold')
        ax.set_xticklabels([f'{int(x)}%' for x in pivot_data.columns], rotation=0)

    plt.tight_layout()
    return fig

## Summary Statistics Table

In [28]:
def create_summary_table(df):
    """Create a comprehensive summary table"""
    summary_data = []

    for model in df['model'].unique():
        model_data = df[df['model'] == model]

        for size in sorted(df['dataset_size'].unique()):
            size_data = model_data[model_data['dataset_size'] == size]

            if not size_data.empty:
                row = size_data.iloc[0]
                summary_data.append({
                    'Model': model,
                    'Dataset %': f"{int(size)}%",
                    'F1 Score': f"{row['f1_score']:.2f}",
                    'Exact Match': f"{row['exact_match']:.2f}",
                    'CO₂ (kg)': f"{row['emissions_kg']:.4f}",
                    'Energy (kWh)': f"{row['energy_consumed_kwh']:.4f}",
                    'Time (hrs)': f"{row['training_time_hours']:.2f}",
                    'F1/CO₂': f"{row['f1_score']/row['emissions_kg']:.2f}",
                    'F1/kWh': f"{row['f1_score']/row['energy_consumed_kwh']:.2f}"
                })

    summary_df = pd.DataFrame(summary_data)

    # Create figure with table
    fig, ax = plt.subplots(figsize=(18, 8))
    ax.axis('tight')
    ax.axis('off')

    table = ax.table(cellText=summary_df.values,
                    colLabels=summary_df.columns,
                    cellLoc='center',
                    loc='center',
                    colWidths=[0.1]*len(summary_df.columns))

    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 2.5)

    # Style header
    for i in range(len(summary_df.columns)):
        table[(0, i)].set_facecolor('#34495e')
        table[(0, i)].set_text_props(weight='bold', color='white')

    # Alternate row colors
    for i in range(1, len(summary_df) + 1):
        for j in range(len(summary_df.columns)):
            if i % 2 == 0:
                table[(i, j)].set_facecolor('#ecf0f1')
            else:
                table[(i, j)].set_facecolor('white')

    plt.title('Comprehensive Model Comparison Summary',
             fontsize=16, fontweight='bold', pad=20)

    return fig, summary_df

## MAIN EXECUTION

In [29]:
def main():
    """Generate all visualizations"""
    print("Loading data...")
    combined_df, gpt2_df, bert_df, distilbert_df = load_data()

    print("\nGenerating visualizations...")

    # 1. Performance Comparison
    print("  📊 Creating performance comparison plots...")
    fig1 = plot_performance_comparison(combined_df)
    fig1.savefig('/content/drive/MyDrive/Carbonemission/Version2/plots/01_performance_comparison.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 2. Emissions Analysis
    print("  🌱 Creating emissions analysis plots...")
    fig2 = plot_emissions_analysis(combined_df)
    fig2.savefig('/content/drive/MyDrive/Carbonemission/Version2/plots/02_emissions_analysis.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 3. Hardware Usage
    print("  💻 Creating hardware usage plots...")
    fig3 = plot_hardware_usage(combined_df)
    fig3.savefig('/content/drive/MyDrive/Carbonemission/Version2/plots/03_hardware_usage.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 4. Performance vs Cost
    print("  ⚖️ Creating performance vs cost trade-off plots...")
    fig4 = plot_performance_vs_cost(combined_df)
    fig4.savefig('/content/drive/MyDrive/Carbonemission/Version2/plots/04_performance_vs_cost.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 5. Comparison Heatmap
    print("  🔥 Creating comparison heatmaps...")
    fig5 = plot_comparison_heatmap(combined_df)
    fig5.savefig('/content/drive/MyDrive/Carbonemission/Version2/plots/05_comparison_heatmap.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 6. Summary Table
    print("  📋 Creating summary table...")
    fig6, summary_df = create_summary_table(combined_df)
    fig6.savefig('/content/drive/MyDrive/Carbonemission/Version2/plots/06_summary_table.png', dpi=300, bbox_inches='tight')
    summary_df.to_csv('model_comparison_summary.csv', index=False)
    plt.close()

    print("\n✅ All visualizations saved!")
    print("\nFiles created:")
    print("  - 01_performance_comparison.png")
    print("  - 02_emissions_analysis.png")
    print("  - 03_hardware_usage.png")
    print("  - 04_performance_vs_cost.png")
    print("  - 05_comparison_heatmap.png")
    print("  - 06_summary_table.png")
    print("  - model_comparison_summary.csv")

    print("\n" + "="*80)
    print("KEY INSIGHTS:")
    print("="*80)

    # Calculate some insights
    best_f1 = combined_df.loc[combined_df['f1_score'].idxmax()]
    most_efficient = combined_df.assign(
        efficiency=combined_df['f1_score']/combined_df['emissions_kg']
    ).loc[combined_df.assign(
        efficiency=combined_df['f1_score']/combined_df['emissions_kg']
    )['efficiency'].idxmax()]

    print(f"🏆 Best F1 Score: {best_f1['model']} with {best_f1['f1_score']:.2f} "
          f"({int(best_f1['dataset_size'])}% dataset)")
    print(f"♻️ Most Carbon Efficient: {most_efficient['model']} "
          f"({int(most_efficient['dataset_size'])}% dataset)")
    print(f"   - F1 per kg CO₂: {most_efficient['f1_score']/most_efficient['emissions_kg']:.2f}")

    return combined_df, summary_df

if __name__ == "__main__":
    combined_df, summary_df = main()
    print("\n✨ Visualization pipeline complete!")

Loading data...

Generating visualizations...
  📊 Creating performance comparison plots...
  🌱 Creating emissions analysis plots...


/tmp/ipython-input-1102713951.py:17: UserWarning: Glyph 8322 (\N{SUBSCRIPT TWO}) missing from font(s) Liberation Sans.
  fig2.savefig('/content/drive/MyDrive/Carbonemission/Version2/plots/02_emissions_analysis.png', dpi=300, bbox_inches='tight')


  💻 Creating hardware usage plots...
  ⚖️ Creating performance vs cost trade-off plots...


/tmp/ipython-input-2135145019.py:51: UserWarning: Glyph 8322 (\N{SUBSCRIPT TWO}) missing from font(s) Liberation Sans.
  plt.tight_layout()
/tmp/ipython-input-1102713951.py:29: UserWarning: Glyph 8322 (\N{SUBSCRIPT TWO}) missing from font(s) Liberation Sans.
  fig4.savefig('/content/drive/MyDrive/Carbonemission/Version2/plots/04_performance_vs_cost.png', dpi=300, bbox_inches='tight')


  🔥 Creating comparison heatmaps...
  📋 Creating summary table...


/tmp/ipython-input-1102713951.py:41: UserWarning: Glyph 8322 (\N{SUBSCRIPT TWO}) missing from font(s) Liberation Sans.
  fig6.savefig('/content/drive/MyDrive/Carbonemission/Version2/plots/06_summary_table.png', dpi=300, bbox_inches='tight')



✅ All visualizations saved!

Files created:
  - 01_performance_comparison.png
  - 02_emissions_analysis.png
  - 03_hardware_usage.png
  - 04_performance_vs_cost.png
  - 05_comparison_heatmap.png
  - 06_summary_table.png
  - model_comparison_summary.csv

KEY INSIGHTS:
🏆 Best F1 Score: BERT with 0.68 (80% dataset)
♻️ Most Carbon Efficient: DistilBERT (25% dataset)
   - F1 per kg CO₂: 129.92

✨ Visualization pipeline complete!
